In [ ]:
# Detecting eye

import cv2   # Importing the cv2 module.
cam = cv2.VideoCapture(0)

detector = cv2.CascadeClassifier('haarcascade_eye.xml')   # Using this haarcascade as a classifier.

while True:
    ret, img = cam.read()   # ret = True if cam is able to read the image.
                            # img captures all the images.
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)   # Converting all the images in grayscale.
    
    eye = detector.detectMultiScale(gray, 1.3, 5)
    
    for (x,y,w,h) in eye:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)    # Drawing the rectangle.
        
    cv2.imshow('frame', img)
    
    if cv2.waitKey(1)==ord('q'):   # Pressing 'q' exits the frame.
        break
        
cam.release()
cv2.destroyAllWindows()